## 1. Import packages

In [ ]:
from bs4 import BeautifulSoup # html을 soup에 넣고 특정 class or data를 가져오는 library
import re
import urllib
import requests
import shutil
import collections
import codecs
import json
from selenium import webdriver

In [2]:
def hangul(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return str(result)

## 2. Input : User_name + logNo

In [3]:
# http://blog.naver.com/PostList.nhn?blogId= *id*
doc = collections.OrderedDict()

move_folder = "C:\\Users\\AjouHCI\\Desktop\\Pic_sample\\Non_Credible"

blog_id= 'dosinam777'
post_No= '221293220120'

url = "http://blog.naver.com/PostView.nhn?blogId="+blog_id+ "&logNo=" + post_No +"&redirect=Dlog&widgetTypeCall=true"

In [4]:
r = requests.get(url)
bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

## 2-1. Input : User_name + page number

In [5]:
# doc = collections.OrderedDict()

# page = 10
# move_folder = "C:\\Users\\AjouHCI\\Desktop\\Pic_sample\\Non_Credible"
# #move_folder = "C:\\Users\\AjouHCI\\Desktop\\Pic_sample\\Credible"

In [6]:
# credible = []
# Non_credible = []

In [7]:
# url = "http://blog.naver.com/PostList.nhn?blogId=" + blog_id + "&currentPage=" + str(page)

In [8]:
# r = requests.get(url)
# bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

## 3. Data_parsing

### 3-1. Get a Title

In [9]:
title = bs.find("h3", {"class": "se_textarea"})
if (title == None):
    title = bs.find("span", {"class": "pcol1 itemSubjectBoldfont"})
if (title != None):
    doc["title"] = title.text.strip()
else:
    doc["title"] = "TITLE ERROR"

### 3-2. Get a Date

In [10]:
date = bs.find("span", {"class": "se_publishDate pcol2 fil5"})

In [11]:
date_text = date.text
date_text = re.sub("\n","",date_text)
date = re.sub("\t","",date_text)

### 3-3. Get a Tag & Content Structure& text

In [12]:
structure = bs.find("div", {"id": "postViewArea"})

if structure == None:
    structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})

structure_tag = structure.find_all(['p','img'])
content_list = []

for i in structure_tag:        
    if str(i)[:2] == "<p":
        content_list.append("<p>")
        check = i.find_all(['span','br'])
        for j in check:
            if str(j)[:2] == '<s':
                if hangul(j.text.strip()) != '':
                    content_list.append(j)
            else:
                content_list.append(j)
        content_list.append("</p>")

    if str(i)[:4] == "<img":
        content_list.append(i)

In [13]:
text_list = []
for i in content_list:
    try:
        text_list.append(i.text)
    except:
        pass

In [14]:
content_structre = "|".join([str(i) for i in content_list])
text = " ".join(list(map(lambda x : x.strip(),list(filter(lambda x: len(x)>1 ,text_list)))))
tag_structure = "|".join(list(map(lambda x: str(x)[:5].strip() +'>' if str(x)[:5][-1] != '>' else str(x)[:5] ,content_list)))

### 3-4. Get a categry_list

In [15]:
driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
driver.get(url)
driver.implicitly_wait(3)
category = driver.find_element_by_class_name("cm-body")
bs = BeautifulSoup(category.text.encode("utf-8"), "html.parser")

total_post_count = int(re.sub('\D',"",bs.text.split("\n")[0].split(" ")[1]))
category = bs.text.split("\n")[1:]
category_count = len(category)

driver.close()

## 4. Save  a file to json

In [16]:
save_list = []

In [17]:
doc['title'] = title.text
doc['tag_structure'] = tag_structure
doc['content'] = content_structre
doc['text'] = text
doc['date'] = date
doc['total_post_count'] = total_post_count
doc['category'] = category
doc['category_count'] = category_count

save_list.append(dict(doc))

In [18]:
with open('Crawling_text.json', 'w') as f:
    f.write(json.dumps(save_list))

## Read json_file

In [19]:
with open('Crawling_text.json', 'r') as read_file:
    loaded_dictionaries = json.loads(read_file.read())

In [20]:
loaded_dictionaries

[{'category': ['창업정보',
   '창업아이템소개',
   '성공창업노하우',
   '맛집,멋집',
   'IT.컴퓨터',
   '재테크',
   '사진여행',
   '상품리뷰',
   '라이프',
   '모임 생활정보 책자',
   '서울 생활정보',
   '인천 생활정보',
   '부천 생활정보',
   '대전 생활정보',
   '세종 생활정보',
   '전주 생활정보',
   '부산 생활책자'],
  'category_count': 17,
  'content': '<p>|<span class="se_fs_T2"><b>대전갈마동맛집 독도새우전문점 황금새우나라 </b><br/></span>|<br/>|<span class="se_fs_T1" style="color: rgb(241, 47, 34);"><b>1주년기념 할인행사 </b></span>|<span class="se_fs_T2"><b>하네요~~~~~~</b><br/></span>|<br/>|<span class="se_fs_T3"><b><b>대전독도새우,갈마동꽃새우,대전닭새우,독도새우가격,대전도하새우,대전독도새우전문점,대전새우전문점,새우코스요리전문점,</b></b><br/></span>|<br/>|<span class="se_fs_T3"><b><b>대전갈마동맛집,대전독도새우이벤트,새우할인행사,</b></b></span>|<br/>|</p>|<img alt="" class="se_mediaImage __se_img_el" data-attachment-id="IQpsv_YrgPnCwA3TreUMm8cCkd_Y" height="440" id="SEDOC-1528358776638-159508495_image_0_img" src="http://postfiles9.naver.net/MjAxODA2MDVfMjMw/MDAxNTI4MTcyNzI2MjY0.76IM_l0SXLvM-U_jtkHHi6xvaGDTDoZSnufxJ8kafjMg.uaFHja7xvrqTjOYYlf7sKwWpWpQNd5sqvDtC29fJx

## 5. All Imgs Download

In [21]:
move_folder = "C:\\Users\\AjouHCI\\Desktop\\Pic_sample\\Non_Credible"

In [22]:
bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
imgs = bs.find_all('img',class_='se_mediaImage __se_img_el')
for num,img in enumerate(imgs):
    url = str(img).split('src=')[1][1:].split(" ")[0].split("\"")[0]
    savename =  hangul(title.text)[:10] + str(num) + ".jpg"
    urllib.request.urlretrieve(url,savename)
    shutil.move(savename,move_folder)